# Cell-cell connection estimation

The estimation aims to process on spatially resolved ligand-receptors based on SPC and calculate cell cell interaction strength.

> We modified [NICHES](https://github.com/msraredon/NICHES) for the 3D data calculation.

First import all necessary library and custom modified NICHES_3D.R library

In [ ]:
library(Seurat)
library(SeuratObject)
library(SeuratDisk)
library(dplyr)
library(tidyr)
library(reshape2)
library(NICHES)
library(stringr)

source("./NICHES_3D.R")

Read in RDS formated Seurat object which include **x**, **y**, **z** 3D coordinates in obj$meta.data

In [ ]:
obj <- readRDS('WT.rds')

To prepare the ligand-receptor pairs for planarian, we used blast hit to [CellChat human ligand-receptor pairs](https://github.com/jinworks/CellChat).

In [ ]:
custom_lrdb <- read.table('./CellChatDB.human.interaction.planarian.txt', sep='\t', header=T)
custom_lrdb <- custom_lrdb[c('gene_id.ligand', 'gene_id.receptor')]
custom_lrdb <- na.omit(custom_lrdb)
custom_lrdb <- custom_lrdb[!duplicated(custom_lrdb), ]

Then run the NICHES pipeline with spatial cell to cell mode.

In [ ]:
results <- RunNICHES(obj,
                     assay='SCT',
                     LR.database="custom",
                     min.cells.per.ident = 3,
                     min.cells.per.gene = 3,
                     meta.data.to.map = colnames(obj@meta.data),
                     position.x = 'x',
                     position.y = 'y',
                     position.z = 'z',
                     cell_types = 'spc_cluster', 
                     custom_LR_database = custom_lrdb, 
                     k = NULL,
                     rad.set = 60, # median_mindis * 5, five layer cells
                     blend = 'mean',
                     CellToCell = F,
                     CellToSystem = F,
                     SystemToCell = F,
                     CellToCellSpatial = T,
                     CellToNeighborhood = F,
                     NeighborhoodToCell = F,
                     output_format = "seurat"
                     )

In [ ]:
obj.result <- results$CellToCellSpatial
obj.loom <- as.loom(obj.result, filename = 'WT_niches.loom', overwrite = T)
obj.loom$close_all()

This will generate a matrix in loom format with lr pairs as feature and cell pairs as obs. With the calculated interaction strength and cell location, connections among cells were constucted with cells as nodes and sender-reseiver relationships as edges. 

The direction of the connection was calculated as the angle of each edge in the 3D space, which was reflected by the orientation of the edge. The entropy of the connection was estimated based on the degree of randomness of the connection orientations. The entropy was used to infer the region in which highly synergistic cell signaling activities occurred during the remodeling of regeneration following: 
 
$$
Pr⁡[X=s] ≅ freq(s) = \sum_{X=s} X_i \div \sum_{all directions} X_i
$$
$$
estimate(H(x)) = - \sum_{all directions} freq(s) * log⁡(freq(s)) .
$$